In [1]:
import requests
from bs4 import BeautifulSoup
import json
import csv
import torch
from transformers import BertTokenizer, BertModel
import torch.nn as nn
import re
import pandas as pd

# Initialize the BERTSUM model and tokenizer
class BERTSUM(nn.Module):
    def __init__(self, model_name='bert-base-uncased'):
        super(BERTSUM, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.linear = nn.Linear(self.bert.config.hidden_size, 1)
        self.softmax = nn.Softmax(dim=1)  # Per word softmax for importance

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        word_output = outputs.last_hidden_state  # Word token representations
        scores = self.linear(word_output)
        scores = self.softmax(scores)
        return scores.squeeze(-1)

# Load pre-trained model and tokenizer
model = BERTSUM()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to extract most important sentences based on word importance scores
def summarize_text(text, model, tokenizer, num_sentences=1):
    paragraphs = text.split('\n')
    paragraphs = [p for p in paragraphs if p.strip()]  # Hapus paragraf kosong
    important_sentences = []
    
    for paragraph in paragraphs:
        sentences = re.split(r'(?<=[.!?]) +', paragraph)  # Pisah paragraf menjadi kalimat
        paragraph_summary = []

        for sentence in sentences:
            inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
            
            # Hapus token_type_ids jika ada
            if 'token_type_ids' in inputs:
                del inputs['token_type_ids']
            
            tokenized_words = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
            
            with torch.no_grad():
                scores = model(**inputs)
            
            # Cek jika scores tidak kosong
            if scores.numel() > 0:  # Memastikan tensor tidak kosong
                k = min(5, scores.size(0))  # Set k tidak lebih besar dari ukuran tensor
                top_word_indices = torch.topk(scores.squeeze(), k=k).indices
                top_words = [tokenized_words[i.item()] for i in top_word_indices if tokenized_words[i.item()] not in tokenizer.all_special_tokens]

                # Jika kalimat memiliki kata-kata penting, tambahkan ke ringkasan paragraf
                if len(top_words) > 0:
                    paragraph_summary.append(sentence)  # Menambahkan kalimat ke ringkasan paragraf

            else:
                print("Scores tensor is empty. Skipping this sentence.")

        # Kombinasikan kalimat paling penting dari setiap paragraf
        if paragraph_summary:
            important_sentences.append(paragraph_summary[0])  # Ambil kalimat paling penting dari setiap paragraf

    # Gabungkan kalimat yang dipilih menjadi ringkasan singkat
    summary = ' '.join(important_sentences)
    return summary

# Function to scrape website and summarize
def scrape_website(url, title_tag, title_class, date_tag, date_class, content_tag, content_class):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    title = soup.find(title_tag, class_=title_class).get_text(strip=True)
    date = soup.find(date_tag, class_=date_class).get_text(strip=True)
    content = soup.find(content_tag, class_=content_class).get_text(strip=True)

    # Automatically summarize the content using BERTSUM
    summary = summarize_text(content, model, tokenizer, num_sentences=1)  # Adjust to only 1 sentence per paragraph

    return {'Title': title, 'Date': date, 'Summary': summary}

# URL and HTML tag details for the website
websites = [
    {
        'url': 'https://cryptoharian.com/top-3-marketplace-bitcoin-nft-yang-harus-diketahui-semua-orang/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/sec-diperkirakan-bakal-setujui-etf-ethereum-spot-dalam-seminggu-kedepan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/3-kripto-yang-harus-dijual-karena-ada-berita-tidak-bagus/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pendiri-diamondswap-saya-masih-bullish-terhadap-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ingin-mulai-perjalanan-investasi-kripto-simak-3-tips-dari-bos-binance-berikut/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/investor-berbondong-bondong-ke-ww3shiba-saat-dogecoin-dan-ondo-berjuang-untuk-pulih/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-token-yang-harus-diwaspadai-unlock-besar-yang-akan-datang/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/blockdag-dev-release-59-peningkatan-x1-miner-dan-blockchain-explorer-memacu-momentum-presale-di-tengah-hype-giveaway-us-2-juta/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rekt-capital-ini-kesempatan-terakhir-untuk-beli-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/presale-ethereum-terbaik-tahun-2024-untuk-menjadi-jutawan-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/avail-digadang-jadi-pilar-utama-di-ekosistem-blockchain-modular/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/kreditur-mt-gox-mulai-terima-kompensasi-investor-kripto-wajib-hati-hati/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ancaman-tiga-crypto-pepe-meroket-746-flow-stabil-dan-ww3-shiba-membuat-entry-yang-luar-biasa/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pasar-koin-meme-diisukan-bangkit-5-aset-ini-jadi-unggulan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-bergerak-naik-menjelang-akhir-bulan-pertanda-juli-positif/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/review-blockdag-apakah-ini-akan-jadi-hal-besar-berikutnya-di-crypto-5-pertanyaan-terjawab/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/mengapa-pasar-kripto-tak-ikuti-tren-positif-ekonomi-as/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-kembali-anjlok-tren-bearish-mulai-ancam-investor/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/dalam-12-jam-berikut-rekap-sejumlah-informasi-penting-di-pasar-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/akankah-keadaan-berubah-untuk-bitcoin-pepe-melonjak-tapi-masih-tertinggal-dari-memecoin-100x-lipat-baru/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/strategi-di-balik-pertumbuhan-pesar-hamster-kombat/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/review-blockdag-inikah-mahkota-permata-baru-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/4-altcoin-ini-dijamin-memberikan-anda-kemudahan-dalam-hidup/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/analis-beri-sinyal-rebound-intip-titik-terendah-bitcoin-disini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/signal-bottom-sudah-keluar-bagaimana-reaksi-investor-btc/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rangkuman-berita-cryptoharian-alasan-wif-anjlok-hingga-pertumbuhan-pesat-hamster-kombat/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/7-mata-uang-kripto-baru-dengan-potensi-pertumbuhan-tinggi/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/membuka-masa-depan-5-ico-terbaik-2024-telah-hadir/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/analis-ini-prediksi-bitcoin-akan-alami-volatilitas-tinggi-sebelum-beranjak-ke-us-67-000/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/3-skenario-untuk-perjalanan-bitcoin-di-bulan-juli/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/masuki-bulan-juli-jangan-lewatkan-6-agenda-utama-di-minggu-awal/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/fetch-ai-fet-dan-singularity-mengalami-penurunan-besar-tapi-ada-satu-token-kripto-ai-dengan-potensi-besar/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-bergerak-naik-di-awal-bulan-pertanda-pergerakan-positif/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rwa-digadang-jadi-tren-masa-depan-investasi-blockchain/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-hari-ini-naik-ini-3-faktor-pendorongnya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/mengapa-investor-ethereum-ikut-serta-dalam-kereta-hype-raboo/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/mengapa-juli-bisa-menjadi-bulan-yang-positif-untuk-bitcoin-dan-ethereum/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/analisis-5-kripto-dengan-volume-besar-di-awal-pekan-eth-sol-xrp-zro-bnb/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/manager-standard-chatered-bitcoin-bisa-mencapai-us100-000-agustus-ini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/michael-van-de-poppe-prediksi-polkadot-akan-alami-bullish-reversal-listing-kangamoon-kian-dekat-sementara-sol-di-ambang-bull-run/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/masuki-juli-bitcoin-di-persimpangan-jalan-antara-bearish-dan-bullish/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/chainlink-dan-floki-inu-layak-beli-di-bulan-juli-ini-sebabnya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-koin-meme-solana-ini-tunjukkan-performa-luar-biasa-dalam-sebulan-terakhir/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/fomc-minutes-dipublikasi-malam-ini-bagaimana-nasib-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/analis-ini-tetap-yakin-pada-altcoin-meski-sedang-babak-belur-apa-alasannya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/3-token-kripto-ai-terbaik-yang-akan-lampaui-lonjakan-100-persen-pepe-dan-wif-pada-2024/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/halving-bitcoin-mulai-beri-dampak-pada-para-penambang-bitcoin-disinyalir-bakal-alami-lonjakan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/memecoin-vs-altcoin-blockdag-unggul-dengan-aplikasi-x1-mencapai-keberhasilan-presale-1300-persen-mengalahkan-stacks-dan-pepe/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/trader-bitcoin-tetap-bullish-meski-tersungkur-oleh-koreksi/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ini-dia-6-altcoin-yang-bisa-menjadi-shiba-inu-berikutnya-menurut-chatgpt/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/anda-trader-atau-investor-kripto-pastikan-anda-dapat-insentif-ratusan-usdt-dari-flipster/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ini-10-penipuan-kripto-terbesar-di-dunia/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/altcoin-terbaik-2025-blockdag-solana-lainnya-siap-menaklukkan-bull-run/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/altcoin-terbaik-2025-blockdag-solana-lainnya-siap-menaklukkan-bull-run/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bull-run-bitcoin-sudah-berakhir-atau-istirahat-sementara-ini-jawaban-analis/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pemegang-xrp-merangkul-raboo-di-tengah-ketidakpastian-sec-strump-menukik-pasca-debat-presiden/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/solana-spot-etfs-fomo-whale-kripto-jual-bitcoin-btc-untuk-etfswap-etfs-dan-polkadot-dot/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/3-altcoin-ini-bisa-jadi-pelarian-sementara-saat-pasar-anjlok/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/apa-yang-terjadi-di-pasar-bitcoin-saat-ini-menurut-trader-x/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/analis-klaim-bitcoin-akan-crash-lagi-berikut-target-harganya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/mt-gox-mulai-kembalikan-dana-ke-kreditur-investor-siaga-guncangan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rangkuman-berita-cryptoharian-signal-bottom-hingga-3-altcoin-pelarian-saat-bearish/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/dompet-bitcoin-berusia-satu-dekade-kembali-aktif-transfer-besar-terjadi/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/dugaan-analis-terkait-pola-pergerakan-bitcoin-yang-sama-tahun-2019-dan-2024/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ini-dia-faktor-teknis-dan-fundamental-di-balik-harga-bitcoin-yang-anjlok-lagi/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ini-dia-faktor-teknis-dan-fundamental-di-balik-harga-bitcoin-yang-anjlok-lagi/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/newwit-dan-cronos-siap-gebrak-dunia-prediksi-ai-dengan-kampanye-seru-bareng-euro-2024/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/apakah-bitcoin-saat-ini-sudah-capai-titik-terendah/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-masih-stagnan-tiga-altcoin-jadi-primadona-pasar/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/investor-bitcoin-veteran-diversifikasi-portofolionya-untuk-pertama-kali-mengatakan-anda-tidak-boleh-melewatkan-ico-ethereum-ini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/aplikasi-pintu-luncurkan-pintu-pro-platform-crypto-untuk-trader-pro/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/aplikasi-pintu-luncurkan-pintu-pro-platform-crypto-untuk-trader-pro/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/trader-klaim-pasar-bull-bitcoin-belum-berakhir-ini-alasannya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/3-airdrop-dengan-jadwal-pembagian-pekan-ini-debank-pancakeswap-sanctum/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/dimana-dan-kapan-anda-bisa-menjual-koin-hamster-kombat/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/4-platform-defi-bitcoin-di-tahun-2024/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/2-koin-kripto-yang-lagi-oversold-saat-ini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/daftar-5-wallet-crypto-paling-aman-buat-trading-dan-investasi-crypto-kamu/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/3-aplikasi-defi-di-ekosistem-blast-dengan-potensi-airdrop-thruster-juice-finance-hyperlock-finance/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/mengenal-meme-koin-terkenal-water-coin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/jpmorgan-prediksi-pemulihan-besar-untuk-bitcoin-di-bulan-agustus/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/jpmorgan-prediksi-pemulihan-besar-untuk-bitcoin-di-bulan-agustus/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/jpmorgan-prediksi-pemulihan-besar-untuk-bitcoin-di-bulan-agustus/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/jpmorgan-prediksi-pemulihan-besar-untuk-bitcoin-di-bulan-agustus/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/7-alasan-kenapa-bull-bitcoin-belum-selesai-menurut-trader/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ethereum-vs-solana-siapa-yang-akan-menguasai-pasar-defi/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/memahami-airdrop-terbaru-di-ekosistem-ton-dogs-meme-coin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    }, 
    {
        'url': 'https://cryptoharian.com/jerman-sudah-habis-jual-bitcoin-apa-yang-bakal-terjadi-selanjutnya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    }, 
    {
        'url': 'https://cryptoharian.com/pasar-kripto-penuh-adrenaline-ini-deretan-kabar-besar-untuk-anda/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/investor-cosmos-beralih-ke-blockdag-untuk-bonus-referral-10-persen-dan-pendapatan-pasif-miner-x30-saat-harapan-okb-menurun/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/investor-cosmos-beralih-ke-blockdag-untuk-bonus-referral-10-persen-dan-pendapatan-pasif-miner-x30-saat-harapan-okb-menurun/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    }, 
    {
        'url': 'https://cryptoharian.com/apa-yang-akan-terjadi-pada-altcoin-menurut-trader-twitter/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    }, 
    {
        'url': 'https://cryptoharian.com/2-level-yang-harus-dijaga-untuk-bitcoin-saat-ini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    }, 
    {
        'url': 'https://cryptoharian.com/3-dex-ethereum-terbaik-di-tahun-2024/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pepe-coin-dan-dogwifhat-jadi-koin-meme-favorit-di-pasar-berikut-penjelasannya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rangkuman-berita-cryptoharian-mt-gox-mulai-kembalikan-dana-kreditur-hingga-jerman-jual-habis-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rangkuman-berita-cryptoharian-mt-gox-mulai-kembalikan-dana-kreditur-hingga-jerman-jual-habis-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rangkuman-berita-cryptoharian-mt-gox-mulai-kembalikan-dana-kreditur-hingga-jerman-jual-habis-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pasar-kripto-bergejolak-usai-upaya-pembunuhan-trump-di-pennsylvania/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/misteri-peretasan-dmm-bitcoin-jejak-dana-us-305-juta-menuju-grup-lazarus/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/mana-yang-lebih-bagus-antara-ethereum-dan-cardano-menurut-chatgpt/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/3-alasan-bitcoin-bergerak-naik-kembali-ke-62800/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/etf-ethereum-dapat-izin-sec-berikut-tanggal-mainnya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pasar-altcoin-sudah-hampir-bottom-kenaikan-menanti-menurut-analis/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/okx-akan-bekerja-sama-dengan-cellula-untuk-bagikan-nft-cellula-bitcell/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-token-unlock-yang-harus-diwaspadai-arb-axs-ape-nym-astr/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/analisis-teknis-dan-onchain-tunjukkan-bitcoin-alami-pemulihan-dari-koreksi/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/altcoin-di-titik-terendah-analis-veteran-klaim-bakal-ada-pemulihan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/mulai-perusahaan-hingga-prediksi-berikut-hightlight-kripto-dalam-24-jam-terakhir/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-kripto-sidechain-bitcoin-dengan-potensi-pergerakan-positif-stx-ela-savm-bvm-ml/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pemerintah-jerman-akhirnya-angkat-bicara-terkait-keputusan-jual-seluruh-stok-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/10-katalis-utama-yang-bakal-jadi-pendorong-utama-naiknya-bitcoin-menurut-analis/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/etfswap-etfs-siap-menjadi-primadona-baru-di-dunia-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/token-ai-kripto-ethereum-yang-wajib-diperhatikan-di-tahun-2024/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-koin-meme-ini-diprediksi-bikin-investor-cuan-besar-saat-bull-run/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/flipster-dan-bonk-inu-hadirkam-kontes-meme-terheboh-tahun-ini-yuk-ikutan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/sentimen-negatif-dan-penarikan-di-bursa-meningkat-bitcoin-tidak-jadi-pulih/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/panduan-menggunakan-aplikasi-token-unlocks-untuk-strategi-trading/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/tekanan-jual-bitcoin-dari-kreditur-mt-gox-ternyata-lebih-rendah-dari-perkiraan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/jangan-lewatkan-sederet-kabar-terbaru-kripto-mulai-pergerakan-hingga-pembukaan-koin-di-sini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/soal-kelebihan-dan-kekurangan-antara-mata-uang-kripto-vs-fiat-simak-penjelasan-robot-berikut/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/token-hmstr-dari-hamster-kombat-listing-sebentar-lagi-berapa-harganya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/kapan-bull-run-kripto-akan-berakhir-ini-jawaban-lady-of-crypto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-bisa-meroket-ke-us-100-000-pada-2024-menurut-analis-terkenal/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/waspada-pemain-hamster-kombat-jadi-target-skema-phising/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/mt-gox-akan-lunasi-kreditur-ini-sejumlah-catatan-untuk-investor/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/analis-mengatakan-harga-xrp-bisa-melonjak-60-000-persen-tetapi-token-ethereum-ini-adalah-pilihan-yang-lebih-baik/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bisakah-investor-kripto-jadi-milyuner-bermodal-koin-shiba-inu/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rangkuman-berita-kripto-2-koin-meme-favorit-hingga-pemain-hamster-kombat-jadi-target-phising/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/empat-event-penting-yang-akan-mempengaruhi-pasar-kripto-minggu-ini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pemerintah-jerman-rugi-us124-juta-dari-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pembaruan-pasar-kripto-shiba-inu-shib-pepe-dan-rco-finance-melaju-menuju-us-1/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/10-koin-kripto-ini-dianggap-berisiko-oleh-trader-top-ada-apa/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-meroket-kembali-ke-68000-sentimen-positif-membawa-harapan-baru-di-pekan-ini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/etf-ethereum-siap-hadir-apa-artinya-bagi-investor/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/cryptoquant-apakah-bitcoin-sudah-bullish-atau-masih-bearish/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/kisruh-koin-meme-di-amerika-jelang-pemilihan-presiden/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-airdrop-potensial-dalam-waktu-dekat-untuk-para-pejuang-airdrop/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/kreditur-mt-gox-terima-kompensasi-bitcoin-turun/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/etf-ethereum-memulai-perdagangan-dengan-volume-miliaran-dolar/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/dogecoin-dan-ethereum-stagnan-sementara-bitcoin-bersiap-untuk-uji-ulang-resistensi-utama/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/investor-tetap-optimis-meski-dogecoin-alami-penurunan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pergerakan-5-kripto-terbesar-di-ekosistem-ethereum-shib-mnt-rndr-arb-ondo/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-lesu-ini-penyebab-dan-prediksi-bulan-agustus/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/setelah-amerika-ferrari-kini-terima-bitcoin-dan-ethereum-di-eropa/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-aset-kripto-yang-holdernya-sangat-loyal-dan-banyak/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-bergerak-turun-menjelang-publikasi-data-pdb-dan-inflasi-amerika/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/alex-pereira-petarung-top-ufc-bermitra-dengan-blockdag-sebagai-duta-penjualan-awal-targetkan-us-100-juta-di-tengah-proyeksi-xlm-okb/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/langkah-berani-marathon-digital-masuk-investasi-us-100-juta-di-bitcoin-bitcoin-naik/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-bisa-capai-us-180-000-dengan-dukungan-federal-reserve/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/upaya-rebound-doge-dan-render-dikalahkan-oleh-ai-moonshot-raboo/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-terancam-turun-lagi-berikut-analisis-terkini-menurut-ahli/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/mungkinkah-bitcoin-capai-us-110-000-di-tahun-2025-ini-prediksi-dari-meta-ai/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/panduan-menggunakan-coinglass-untuk-melihat-open-interest-saat-trading-aset-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/harga-harga-bitcoin-penting-untuk-bulan-juli-dan-agustus-menurut-analis/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rekor-donasi-kripto-trump-terima-us-4-juta-untuk-kampanye-2024/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/kepala-riset-matrixport-menjelaskan-kenapa-harga-ethereum-turun/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ceo-cryptoquant-klaim-bull-bitcoin-bakal-berlanjut-hingga-tahun-depan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/hadiri-konferensi-bitcoin-di-nashville-berikut-deretan-janji-trump-untuk-aset-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/awas-3-altcoin-ini-terancam-alami-penurunan-hebat/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rangkuman-berita-cryptoharian-jadi-milyuner-bermodal-shiba-inu-hingga-rekor-donasi-kripto-untuk-trump/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/sejumlah-agenda-krusial-bakal-gerakkan-pasar-kripto-dalam-minggu-ini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/chatgpt-4-o-beberkan-alasan-kenapa-blackrock-hanya-fokus-pada-bitcoin-dan-ethereum/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rco-finance-rcof-dan-shiba-inu-shib-bersaing-dalam-peningkatan-laju-pembakaran-akselerasi-harga-segera-terjadi/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-breakout-ini-tanda-tanda-altseason-yang-harus-diperhatikan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-bergerak-naik-di-tengah-sentimen-positif-apakah-69000-akan-kuat/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-alami-penurunan-tajam-saat-upaya-tembus-us-70-000-gagal/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-bakal-raih-lagi-ath-ini-tips-trading-ideal-menurut-analis/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/whale-cardano-dan-bonk-memperhatikan-meme-coin-ai-raboo/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/apa-saja-yang-anda-lewatkan-saat-absen-24-jam-di-pasar-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-token-unlock-yang-harus-diwaspadai-prime-op-yfi-pendle-1inch/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ceo-blockdag-antony-turner-mantan-co-founder-swissone-capital-menargetkan-us-600-juta-mengarahkan-investor-menjauh-dari-tren-sol-dan-dot/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/payout-mt-gox-sudah-berada-di-tangan-para-kreditur-pasar-kripto-siap-berguncang/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ethereum-terperosok-pasca-alami-kenaikan-signifikan-ada-apa/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pemerintahan-rusia-kemungkinan-akan-masuk-ke-ranah-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-kripto-potensial-dalam-kategori-defi-jto-aave-jup-dodo-uma/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/wawancara-ceo-blockdag-dan-tim-ama-tingkatkan-kepercayaan-membuat-roi-kelas-terkemuka-menjadi-mungkin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/jangan-ketinggalan-sejumlah-hal-besar-terjadi-pada-pasar-kripto-12-jam-lalu/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-bull-run-di-depan-mata-analis-ini-urai-kaitannya-dengan-aset-emas/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/koin-meme-iq-capai-kapitalisasi-pasar-us-5-juta-dalam-24-jam-kok-bisa/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-airdrop-potensial-dalam-waktu-dekat-untuk-para-pejuang-airdrop-2/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/wawancara-ceo-ungkap-informasi-utama-lonjakan-presale-us-64-juta-blockdag-wawasan-icp-berita-ton/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/microstrategy-beli-bitcoin-lagi-ini-jumlah-total-kepemilikannya-sekarang/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/apakah-bitcoin-akan-semakin-turun-ini-level-yang-perlu-diwaspadai-investor/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/analisa-shiba-inu-aptos-dan-raboo-coin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ini-sejumlah-analisa-terkait-puncak-harga-bitcoin-selanjutnya-dari-trader-hingga-peneliti/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/apakah-optimism-op-masih-layak-dibeli-melihat-analisis-data-onchain/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/harga-bitcoin-terjun-bebas-analis-peringatkan-potensi-penurunan-lebih-lanjut/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/empat-pengusaha-kripto-terkaya-dan-kerajaan-nya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ekonomi-as-di-ujung-tanduk-ekonom-ini-prediksi-resesi-dan-inflasi-tinggi/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-jatuh-hingga-kisaran-us-60-000-ini-biang-keladinya-menurut-trader-papan-atas/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/token-bertema-anjing-tumpang-tindih-dengan-tren-meme-ai-kelinci-bernama-raboo-sudahkah-anda-ikut/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/binance-sambut-hamster-kombat-antusiasme-meledak-di-pasar-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ai-ungkap-alasan-solana-lebih-bagus-daripada-cardano/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/apakah-ada-kesempatan-pulih-bagi-bitcoin-setelah-anjlok-di-bawah-us-60-000/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/kenapa-harga-bitcoin-turun-ke-53-000-hari-ini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/penambang-bitcoin-kini-dihadapkan-pada-sejumlah-tantangan-baru/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ripple-bersiap-jual-200-juta-xrp-ada-apa/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/3-faktor-penyebab-koreksi-bitcoin-ke-daerah-50-000-dalam-3-hari-berturut-turut/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/seluruh-aset-kripto-ambruk-dalam-hitungan-jam-simak-kilasan-info-nya-di-sini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/harga-bitcoin-btc-turun-di-bawah-us-60-000-diikuti-ripple-xrp-saat-tekanan-bearish-meningkat-pada-token-perusahaan-pembayaran/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bernstein-pasar-kripto-tertekan-tapi-pelakunya-bukan-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/akankah-level-di-bawah-us-50-000-jadi-petaka-untuk-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/apakah-dot-masih-layak-dibeli-melihat-analisis-data-transaksi-polkadot/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/analis-sebut-bitcoin-membentuk-pola-rising-wedge-apa-artinya-bagi-investor/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/whale-kripto-tak-terpengaruh-oleh-penurunan-aliran-keluar-spot-ethereum-pembelian-besar-eth-terus-berlanjut/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/qcp-capital-vs-crypnuevo-dalam-pandangan-terkait-potensi-pergerakan-harga-bitcoin-selanjutnya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/investor-ini-tuding-jepang-dan-amerika-jadi-penyebab-utama-guncangan-hebat-di-pasar-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/apakah-link-masih-layak-dibeli-meneliti-data-transaksi-chainlink/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-turun-di-bawah-us-50-000-simak-pendapat-para-ahli-berikut/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/apa-itu-solayer-proyek-restaking-terbaru-yang-didukung-binance-labs/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/koin-meme-baru-neiro-siap-hebohkan-pasar-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-token-unlock-yang-harus-diwaspadai-gmx-imx-xai-fort-glmr/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rebound-besar-analis-ini-perkirakan-harga-bitcoin-masih-bisa-naik-ke-us-90-000/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/trader-dogecoin-doge-legendaris-mengatakan-token-ethereum-seharga-us-00344-ini-akan-naik/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/arthur-hayes-klaim-target-harga-bitcoin-dan-ethereum-saat-carut-marut-ekonomi-amerika/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/airdrop-hamster-kombat-diisukan-jadi-yang-terbesar-sepanjang-sejarah-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/harga-bitcoin-bergerak-naik-setelah-turun-drastis-semua-volume-beli-berasal-dari-whale-institusional/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/mulai-dari-legalisasi-kripto-di-rusia-hingga-pertemuan-bersama-gedung-putih-ini-deretan-info-kripto-dalam-24-jam/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/kenapa-neiro-mona-dan-shibtc-jadi-primadona-di-kalangan-koin-meme/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pasar-altcoin-di-ambang-pemulihan-menurut-analis-benjamin-cowen/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/saat-pasar-kripto-anjlok-trader-ini-empat-kripto-untuk-tetap-dulang-cuan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rangkuman-berita-cryptoharian-binance-sambut-hamster-kombat-hingga-pasar-altcoin-di-ambang-pemulihan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ekonomi-dunia-diperkirakan-hancur-di-tahun-2026-ini-deretan-penyebabnya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/6-koin-yang-memiliki-agenda-minggu-ini-bitcoin-hingga-sandbox/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bagaimana-harga-bitcoin-ditentukan-ini-jawaban-lengkapnya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/apakah-shiba-inu-bisa-bangkit-kembali-setelah-kejatuhan-besarnya-baru-baru-ini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/youssef-khaoulaj-ahli-metaverse-bersertifikat-memimpin-blockdag-lebih-banyak-tentang-perkiraan-harga-cardano-ada-uniswap-uni/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/volatilitas-berlanjut-menjelang-publikasi-data-inflasi-amerika-bitcoin-turun-ke-bawah-60-000/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/sec-novatech-lakukan-penipuan-kripto-us-650-juta-jerat-ribuan-investor/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/solana-sol-cardano-ada-dan-token-defi-ethereum-eth-ini-tampaknya-akan-bangkit-saat-bear-trap-bitcoin-selesai/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/hamster-kombat-berani-tolak-pemodal-raksasa-ini-alasannya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pekan-ini-diprediksi-jadi-momen-krusial-bagi-bitcoin-saat-harga-sentuh-kisaran-us-57-000/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-token-unlock-yang-harus-diwaspadai-crv-pendle-sand-cyber-strk/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/cpi-amerika-jadi-penentu-arah-harga-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/sejumlah-analis-perkirakan-harga-bitcoin-bakal-capai-nilai-enam-digit/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/banyak-prediksi-meleset-kini-giliran-arthur-hayes-ungkap-kapan-altcoin-akan-berjaya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/indonesia-siap-memimpin-kedaulatan-digital-melalui-bali-blockchain-summit-2024-dan-web3-hackathon/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/harga-bitcoin-kembali-naik-ke-61-000-didukung-oleh-aksi-beli-whale-dan-data-inflasi-yang-lebih-rendah/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-koreksi-5-di-tengah-publikasi-data-ekonomi-as-kesempatan-untuk-untung/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/mengapa-bank-besar-seperti-goldman-sachs-mulai-melirik-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/altcoin-sherpa-bitcoin-berada-di-persimpangan-siap-siap-untuk-pergerakan-besar/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bagaimana-dengan-blockdag-setelah-harga-binance-coin-dan-protokol-near-jatuh/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/smart-money-mengumpulkan-jutaan-dalam-cardano-ada-pepe-dan-shiba-inu-killer-senilai-00344-ini-untuk-keuntungan-4600/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-airdrop-potensial-dalam-untuk-para-pencari-cuan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/kilas-balik-24-jam-terakhir-dalam-perkembangan-terbaru-di-dunia-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-naik-dari-harga-us-58-000-sinyal-bahaya-atau-peluang/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/trader-veteran-yang-membeli-dogecoin-di-awal-mengatakan-token-ethereum-ini-akan-mengalami-kenaikan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ai-ini-prediksi-harga-ethereum-di-penghujung-bulan-berikut-targetnya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bangun-kekayaan-real-estat-anda-dengan-game-clickcity-senilai-us-1-juta-dari-lufina/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/7-alasan-kenapa-bitcoin-anjlok-sebentar-lagi-menurut-analis-berikut/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pasar-kripto-semakin-kemari-semakin-kehilangan-marwah/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rangkuman-berita-cryptoharian-perkiraan-ekonomi-hancur-hingga-prediksi-ethereum-akhir-agustus/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/simak-apa-saja-yang-akan-gerakkan-pasar-kripto-dalam-pekan-ini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/analis-kripto-benjamin-cowen-yakin-dominasi-bitcoin-akan-capai-60-persen/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/whale-bergerak-litecoin-dan-dogecoin-jadi-fokus-utama/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/hati-hati-50-persen-koin-meme-mengalami-kerugian/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-jatuh-di-bawah-60-000-apa-arti-dari-koreksi-ini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
]

all_articles = []

# Collect and summarize data from each website
for website in websites:
    article = scrape_website(
        website['url'], 
        website['title_tag'], 
        website['title_class'], 
        website['date_tag'], 
        website['date_class'], 
        website['content_tag'], 
        website['content_class']
    )
    all_articles.append(article)

# Save summarized data to CSV file
csv_file = './preprocessed/cryptoharian.csv'
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['Title', 'Date', 'Summary'])
    writer.writeheader()
    for article in all_articles:
        writer.writerow(article)

# Print the contents of the CSV file
print(f"Data has been saved to {csv_file}")

df = pd.read_csv(csv_file)
print(df)

Data has been saved to ./preprocessed/cryptoharian.csv
                                                 Title              Date  \
0    Top 3 Marketplace Bitcoin NFT yang Harus Diket...     Juni 26, 2024   
1    SEC Diperkirakan Bakal Setujui ETF Ethereum Sp...     Juni 27, 2024   
2    3 Kripto Yang Harus Dijual Karena Ada Berita T...     Juni 27, 2024   
3    Pendiri DiamondSwap: Saya Masih Bullish Terhad...     Juni 27, 2024   
4    Ingin Mulai Perjalanan Investasi Kripto? Simak...     Juni 27, 2024   
..                                                 ...               ...   
253  Simak Apa Saja yang Akan Gerakkan Pasar Kripto...  Agustus 19, 2024   
254  Analis Kripto Benjamin Cowen Yakin Dominasi Bi...  Agustus 19, 2024   
255  Whale Bergerak, Litecoin dan Dogecoin Jadi Fok...  Agustus 19, 2024   
256  Hati-Hati! 50 Persen Koin Meme Mengalami Kerugian  Agustus 19, 2024   
257  Bitcoin Jatuh di Bawah $60.000: Apa Arti dari ...  Agustus 19, 2024   

                                

In [1]:
from bs4 import BeautifulSoup
import requests
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
import torch.nn as nn
import re
import csv
import pandas as pd

# Initialize the BERTSUM model and tokenizer
class BERTSUM(nn.Module):
    def __init__(self, model_name='bert-base-uncased'):
        super(BERTSUM, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.linear = nn.Linear(self.bert.config.hidden_size, 1)
        self.softmax = nn.Softmax(dim=1)  # Per word softmax for importance

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        word_output = outputs.last_hidden_state  # Word token representations
        scores = self.linear(word_output)
        scores = self.softmax(scores)
        return scores.squeeze(-1)

# Load pre-trained model and tokenizer
model = BERTSUM()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Fungsi untuk ekstraksi embedding kalimat menggunakan BERT
def get_sentence_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors="pt")
    outputs = model.bert(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.detach().numpy().flatten()

# Fungsi untuk melakukan rangkuman ekstraktif berbasis cosine similarity
def extractive_summary(text, ratio=0.7):
    sentences = text.split('. ')
    sentence_embeddings = [get_sentence_embedding(sentence) for sentence in sentences]

    # Menghitung similaritas kosinus antar kalimat
    similarities = cosine_similarity(sentence_embeddings)
    sentence_scores = similarities.sum(axis=1)

    # Menentukan jumlah kalimat berdasarkan ratio
    top_n = max(1, int(len(sentences) * ratio))
    top_sentence_indices = np.argsort(sentence_scores)[-top_n:]
    top_sentence_indices.sort()

    summary = '. '.join([sentences[i] for i in top_sentence_indices])
    return summary

# Function to extract important sentences based on word importance scores using BERTSUM
def summarize_text(text, model, tokenizer, num_sentences=1):
    paragraphs = text.split('\n')
    paragraphs = [p for p in paragraphs if p.strip()]
    important_sentences = []
    
    for paragraph in paragraphs:
        sentences = re.split(r'(?<=[.!?]) +', paragraph)
        paragraph_summary = []

        for sentence in sentences:
            inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
            
            if 'token_type_ids' in inputs:
                del inputs['token_type_ids']
            
            tokenized_words = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
            
            with torch.no_grad():
                scores = model(**inputs)
            
            if scores.numel() > 0:
                k = min(5, scores.size(0))
                top_word_indices = torch.topk(scores.squeeze(), k=k).indices
                top_words = [tokenized_words[i.item()] for i in top_word_indices if tokenized_words[i.item()] not in tokenizer.all_special_tokens]

                if len(top_words) > 0:
                    paragraph_summary.append(sentence)

            else:
                print("Scores tensor is empty. Skipping this sentence.")

        if paragraph_summary:
            important_sentences.append(paragraph_summary[0])

    summary = ' '.join(important_sentences)
    return summary

# Function to scrape website and summarize
def scrape_website(url, title_tag, title_class, date_tag, date_class, content_tag, content_class, summary_type='bert', ratio=0.7):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    title_element = soup.find(title_tag, class_=title_class)
    date_element = soup.find(date_tag, class_=date_class)
    content_element = soup.find(content_tag, class_=content_class)

    title = title_element.get_text(strip=True) if title_element else "No Title Found"
    date = date_element.get_text(strip=True) if date_element else "No Date Found"
    content = content_element.get_text(strip=True) if content_element else "No Content Found"

    # Memilih metode rangkuman berdasarkan summary_type
    if summary_type == 'bert':
        summary = summarize_text(content, model, tokenizer, num_sentences=1)
    elif summary_type == 'cosine':
        summary = extractive_summary(content, ratio=0.7)
    else:
        raise ValueError("Invalid summary_type. Choose either 'bert' or 'cosine'.")

    return {'Title': title, 'Date': date, 'Summary': summary}

# URL and HTML tag details for the website
websites = [
    {
        'url': 'https://cryptoharian.com/top-3-marketplace-bitcoin-nft-yang-harus-diketahui-semua-orang/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/sec-diperkirakan-bakal-setujui-etf-ethereum-spot-dalam-seminggu-kedepan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/3-kripto-yang-harus-dijual-karena-ada-berita-tidak-bagus/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pendiri-diamondswap-saya-masih-bullish-terhadap-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ingin-mulai-perjalanan-investasi-kripto-simak-3-tips-dari-bos-binance-berikut/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/investor-berbondong-bondong-ke-ww3shiba-saat-dogecoin-dan-ondo-berjuang-untuk-pulih/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-token-yang-harus-diwaspadai-unlock-besar-yang-akan-datang/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/blockdag-dev-release-59-peningkatan-x1-miner-dan-blockchain-explorer-memacu-momentum-presale-di-tengah-hype-giveaway-us-2-juta/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rekt-capital-ini-kesempatan-terakhir-untuk-beli-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/presale-ethereum-terbaik-tahun-2024-untuk-menjadi-jutawan-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/avail-digadang-jadi-pilar-utama-di-ekosistem-blockchain-modular/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/kreditur-mt-gox-mulai-terima-kompensasi-investor-kripto-wajib-hati-hati/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ancaman-tiga-crypto-pepe-meroket-746-flow-stabil-dan-ww3-shiba-membuat-entry-yang-luar-biasa/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pasar-koin-meme-diisukan-bangkit-5-aset-ini-jadi-unggulan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-bergerak-naik-menjelang-akhir-bulan-pertanda-juli-positif/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/review-blockdag-apakah-ini-akan-jadi-hal-besar-berikutnya-di-crypto-5-pertanyaan-terjawab/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/mengapa-pasar-kripto-tak-ikuti-tren-positif-ekonomi-as/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-kembali-anjlok-tren-bearish-mulai-ancam-investor/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/dalam-12-jam-berikut-rekap-sejumlah-informasi-penting-di-pasar-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/akankah-keadaan-berubah-untuk-bitcoin-pepe-melonjak-tapi-masih-tertinggal-dari-memecoin-100x-lipat-baru/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/strategi-di-balik-pertumbuhan-pesar-hamster-kombat/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/review-blockdag-inikah-mahkota-permata-baru-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/4-altcoin-ini-dijamin-memberikan-anda-kemudahan-dalam-hidup/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/analis-beri-sinyal-rebound-intip-titik-terendah-bitcoin-disini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/signal-bottom-sudah-keluar-bagaimana-reaksi-investor-btc/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rangkuman-berita-cryptoharian-alasan-wif-anjlok-hingga-pertumbuhan-pesat-hamster-kombat/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/7-mata-uang-kripto-baru-dengan-potensi-pertumbuhan-tinggi/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/membuka-masa-depan-5-ico-terbaik-2024-telah-hadir/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/analis-ini-prediksi-bitcoin-akan-alami-volatilitas-tinggi-sebelum-beranjak-ke-us-67-000/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/3-skenario-untuk-perjalanan-bitcoin-di-bulan-juli/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/masuki-bulan-juli-jangan-lewatkan-6-agenda-utama-di-minggu-awal/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/fetch-ai-fet-dan-singularity-mengalami-penurunan-besar-tapi-ada-satu-token-kripto-ai-dengan-potensi-besar/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-bergerak-naik-di-awal-bulan-pertanda-pergerakan-positif/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rwa-digadang-jadi-tren-masa-depan-investasi-blockchain/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-hari-ini-naik-ini-3-faktor-pendorongnya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/mengapa-investor-ethereum-ikut-serta-dalam-kereta-hype-raboo/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/mengapa-juli-bisa-menjadi-bulan-yang-positif-untuk-bitcoin-dan-ethereum/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/analisis-5-kripto-dengan-volume-besar-di-awal-pekan-eth-sol-xrp-zro-bnb/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/manager-standard-chatered-bitcoin-bisa-mencapai-us100-000-agustus-ini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/michael-van-de-poppe-prediksi-polkadot-akan-alami-bullish-reversal-listing-kangamoon-kian-dekat-sementara-sol-di-ambang-bull-run/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/masuki-juli-bitcoin-di-persimpangan-jalan-antara-bearish-dan-bullish/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/chainlink-dan-floki-inu-layak-beli-di-bulan-juli-ini-sebabnya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-koin-meme-solana-ini-tunjukkan-performa-luar-biasa-dalam-sebulan-terakhir/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/fomc-minutes-dipublikasi-malam-ini-bagaimana-nasib-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/analis-ini-tetap-yakin-pada-altcoin-meski-sedang-babak-belur-apa-alasannya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/3-token-kripto-ai-terbaik-yang-akan-lampaui-lonjakan-100-persen-pepe-dan-wif-pada-2024/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/halving-bitcoin-mulai-beri-dampak-pada-para-penambang-bitcoin-disinyalir-bakal-alami-lonjakan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/memecoin-vs-altcoin-blockdag-unggul-dengan-aplikasi-x1-mencapai-keberhasilan-presale-1300-persen-mengalahkan-stacks-dan-pepe/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/trader-bitcoin-tetap-bullish-meski-tersungkur-oleh-koreksi/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ini-dia-6-altcoin-yang-bisa-menjadi-shiba-inu-berikutnya-menurut-chatgpt/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/anda-trader-atau-investor-kripto-pastikan-anda-dapat-insentif-ratusan-usdt-dari-flipster/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ini-10-penipuan-kripto-terbesar-di-dunia/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/altcoin-terbaik-2025-blockdag-solana-lainnya-siap-menaklukkan-bull-run/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/altcoin-terbaik-2025-blockdag-solana-lainnya-siap-menaklukkan-bull-run/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bull-run-bitcoin-sudah-berakhir-atau-istirahat-sementara-ini-jawaban-analis/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pemegang-xrp-merangkul-raboo-di-tengah-ketidakpastian-sec-strump-menukik-pasca-debat-presiden/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/solana-spot-etfs-fomo-whale-kripto-jual-bitcoin-btc-untuk-etfswap-etfs-dan-polkadot-dot/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/3-altcoin-ini-bisa-jadi-pelarian-sementara-saat-pasar-anjlok/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/apa-yang-terjadi-di-pasar-bitcoin-saat-ini-menurut-trader-x/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/analis-klaim-bitcoin-akan-crash-lagi-berikut-target-harganya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/mt-gox-mulai-kembalikan-dana-ke-kreditur-investor-siaga-guncangan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rangkuman-berita-cryptoharian-signal-bottom-hingga-3-altcoin-pelarian-saat-bearish/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/dompet-bitcoin-berusia-satu-dekade-kembali-aktif-transfer-besar-terjadi/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/dugaan-analis-terkait-pola-pergerakan-bitcoin-yang-sama-tahun-2019-dan-2024/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ini-dia-faktor-teknis-dan-fundamental-di-balik-harga-bitcoin-yang-anjlok-lagi/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ini-dia-faktor-teknis-dan-fundamental-di-balik-harga-bitcoin-yang-anjlok-lagi/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/newwit-dan-cronos-siap-gebrak-dunia-prediksi-ai-dengan-kampanye-seru-bareng-euro-2024/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/apakah-bitcoin-saat-ini-sudah-capai-titik-terendah/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-masih-stagnan-tiga-altcoin-jadi-primadona-pasar/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/investor-bitcoin-veteran-diversifikasi-portofolionya-untuk-pertama-kali-mengatakan-anda-tidak-boleh-melewatkan-ico-ethereum-ini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/aplikasi-pintu-luncurkan-pintu-pro-platform-crypto-untuk-trader-pro/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/aplikasi-pintu-luncurkan-pintu-pro-platform-crypto-untuk-trader-pro/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/trader-klaim-pasar-bull-bitcoin-belum-berakhir-ini-alasannya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',        
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/3-airdrop-dengan-jadwal-pembagian-pekan-ini-debank-pancakeswap-sanctum/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/dimana-dan-kapan-anda-bisa-menjual-koin-hamster-kombat/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/4-platform-defi-bitcoin-di-tahun-2024/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/2-koin-kripto-yang-lagi-oversold-saat-ini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/daftar-5-wallet-crypto-paling-aman-buat-trading-dan-investasi-crypto-kamu/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/3-aplikasi-defi-di-ekosistem-blast-dengan-potensi-airdrop-thruster-juice-finance-hyperlock-finance/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/mengenal-meme-koin-terkenal-water-coin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/jpmorgan-prediksi-pemulihan-besar-untuk-bitcoin-di-bulan-agustus/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/jpmorgan-prediksi-pemulihan-besar-untuk-bitcoin-di-bulan-agustus/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/jpmorgan-prediksi-pemulihan-besar-untuk-bitcoin-di-bulan-agustus/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/jpmorgan-prediksi-pemulihan-besar-untuk-bitcoin-di-bulan-agustus/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/7-alasan-kenapa-bull-bitcoin-belum-selesai-menurut-trader/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ethereum-vs-solana-siapa-yang-akan-menguasai-pasar-defi/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/memahami-airdrop-terbaru-di-ekosistem-ton-dogs-meme-coin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    }, 
    {
        'url': 'https://cryptoharian.com/jerman-sudah-habis-jual-bitcoin-apa-yang-bakal-terjadi-selanjutnya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    }, 
    {
        'url': 'https://cryptoharian.com/pasar-kripto-penuh-adrenaline-ini-deretan-kabar-besar-untuk-anda/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/investor-cosmos-beralih-ke-blockdag-untuk-bonus-referral-10-persen-dan-pendapatan-pasif-miner-x30-saat-harapan-okb-menurun/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/investor-cosmos-beralih-ke-blockdag-untuk-bonus-referral-10-persen-dan-pendapatan-pasif-miner-x30-saat-harapan-okb-menurun/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    }, 
    {
        'url': 'https://cryptoharian.com/apa-yang-akan-terjadi-pada-altcoin-menurut-trader-twitter/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    }, 
    {
        'url': 'https://cryptoharian.com/2-level-yang-harus-dijaga-untuk-bitcoin-saat-ini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    }, 
    {
        'url': 'https://cryptoharian.com/3-dex-ethereum-terbaik-di-tahun-2024/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pepe-coin-dan-dogwifhat-jadi-koin-meme-favorit-di-pasar-berikut-penjelasannya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rangkuman-berita-cryptoharian-mt-gox-mulai-kembalikan-dana-kreditur-hingga-jerman-jual-habis-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rangkuman-berita-cryptoharian-mt-gox-mulai-kembalikan-dana-kreditur-hingga-jerman-jual-habis-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rangkuman-berita-cryptoharian-mt-gox-mulai-kembalikan-dana-kreditur-hingga-jerman-jual-habis-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pasar-kripto-bergejolak-usai-upaya-pembunuhan-trump-di-pennsylvania/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/misteri-peretasan-dmm-bitcoin-jejak-dana-us-305-juta-menuju-grup-lazarus/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/mana-yang-lebih-bagus-antara-ethereum-dan-cardano-menurut-chatgpt/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/3-alasan-bitcoin-bergerak-naik-kembali-ke-62800/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/etf-ethereum-dapat-izin-sec-berikut-tanggal-mainnya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pasar-altcoin-sudah-hampir-bottom-kenaikan-menanti-menurut-analis/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/okx-akan-bekerja-sama-dengan-cellula-untuk-bagikan-nft-cellula-bitcell/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-token-unlock-yang-harus-diwaspadai-arb-axs-ape-nym-astr/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/analisis-teknis-dan-onchain-tunjukkan-bitcoin-alami-pemulihan-dari-koreksi/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/altcoin-di-titik-terendah-analis-veteran-klaim-bakal-ada-pemulihan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/mulai-perusahaan-hingga-prediksi-berikut-hightlight-kripto-dalam-24-jam-terakhir/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-kripto-sidechain-bitcoin-dengan-potensi-pergerakan-positif-stx-ela-savm-bvm-ml/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pemerintah-jerman-akhirnya-angkat-bicara-terkait-keputusan-jual-seluruh-stok-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/10-katalis-utama-yang-bakal-jadi-pendorong-utama-naiknya-bitcoin-menurut-analis/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/etfswap-etfs-siap-menjadi-primadona-baru-di-dunia-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/token-ai-kripto-ethereum-yang-wajib-diperhatikan-di-tahun-2024/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-koin-meme-ini-diprediksi-bikin-investor-cuan-besar-saat-bull-run/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/flipster-dan-bonk-inu-hadirkam-kontes-meme-terheboh-tahun-ini-yuk-ikutan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/sentimen-negatif-dan-penarikan-di-bursa-meningkat-bitcoin-tidak-jadi-pulih/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/panduan-menggunakan-aplikasi-token-unlocks-untuk-strategi-trading/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/tekanan-jual-bitcoin-dari-kreditur-mt-gox-ternyata-lebih-rendah-dari-perkiraan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/jangan-lewatkan-sederet-kabar-terbaru-kripto-mulai-pergerakan-hingga-pembukaan-koin-di-sini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/soal-kelebihan-dan-kekurangan-antara-mata-uang-kripto-vs-fiat-simak-penjelasan-robot-berikut/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/token-hmstr-dari-hamster-kombat-listing-sebentar-lagi-berapa-harganya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/kapan-bull-run-kripto-akan-berakhir-ini-jawaban-lady-of-crypto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-bisa-meroket-ke-us-100-000-pada-2024-menurut-analis-terkenal/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/waspada-pemain-hamster-kombat-jadi-target-skema-phising/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/mt-gox-akan-lunasi-kreditur-ini-sejumlah-catatan-untuk-investor/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/analis-mengatakan-harga-xrp-bisa-melonjak-60-000-persen-tetapi-token-ethereum-ini-adalah-pilihan-yang-lebih-baik/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bisakah-investor-kripto-jadi-milyuner-bermodal-koin-shiba-inu/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rangkuman-berita-kripto-2-koin-meme-favorit-hingga-pemain-hamster-kombat-jadi-target-phising/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/empat-event-penting-yang-akan-mempengaruhi-pasar-kripto-minggu-ini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pemerintah-jerman-rugi-us124-juta-dari-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pembaruan-pasar-kripto-shiba-inu-shib-pepe-dan-rco-finance-melaju-menuju-us-1/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/10-koin-kripto-ini-dianggap-berisiko-oleh-trader-top-ada-apa/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-meroket-kembali-ke-68000-sentimen-positif-membawa-harapan-baru-di-pekan-ini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/etf-ethereum-siap-hadir-apa-artinya-bagi-investor/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/cryptoquant-apakah-bitcoin-sudah-bullish-atau-masih-bearish/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/kisruh-koin-meme-di-amerika-jelang-pemilihan-presiden/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-airdrop-potensial-dalam-waktu-dekat-untuk-para-pejuang-airdrop/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/kreditur-mt-gox-terima-kompensasi-bitcoin-turun/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/etf-ethereum-memulai-perdagangan-dengan-volume-miliaran-dolar/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/dogecoin-dan-ethereum-stagnan-sementara-bitcoin-bersiap-untuk-uji-ulang-resistensi-utama/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/investor-tetap-optimis-meski-dogecoin-alami-penurunan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pergerakan-5-kripto-terbesar-di-ekosistem-ethereum-shib-mnt-rndr-arb-ondo/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-lesu-ini-penyebab-dan-prediksi-bulan-agustus/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/setelah-amerika-ferrari-kini-terima-bitcoin-dan-ethereum-di-eropa/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-aset-kripto-yang-holdernya-sangat-loyal-dan-banyak/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-bergerak-turun-menjelang-publikasi-data-pdb-dan-inflasi-amerika/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/alex-pereira-petarung-top-ufc-bermitra-dengan-blockdag-sebagai-duta-penjualan-awal-targetkan-us-100-juta-di-tengah-proyeksi-xlm-okb/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/langkah-berani-marathon-digital-masuk-investasi-us-100-juta-di-bitcoin-bitcoin-naik/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-bisa-capai-us-180-000-dengan-dukungan-federal-reserve/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/upaya-rebound-doge-dan-render-dikalahkan-oleh-ai-moonshot-raboo/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-terancam-turun-lagi-berikut-analisis-terkini-menurut-ahli/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/mungkinkah-bitcoin-capai-us-110-000-di-tahun-2025-ini-prediksi-dari-meta-ai/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/panduan-menggunakan-coinglass-untuk-melihat-open-interest-saat-trading-aset-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/harga-harga-bitcoin-penting-untuk-bulan-juli-dan-agustus-menurut-analis/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rekor-donasi-kripto-trump-terima-us-4-juta-untuk-kampanye-2024/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/kepala-riset-matrixport-menjelaskan-kenapa-harga-ethereum-turun/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ceo-cryptoquant-klaim-bull-bitcoin-bakal-berlanjut-hingga-tahun-depan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/hadiri-konferensi-bitcoin-di-nashville-berikut-deretan-janji-trump-untuk-aset-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/awas-3-altcoin-ini-terancam-alami-penurunan-hebat/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rangkuman-berita-cryptoharian-jadi-milyuner-bermodal-shiba-inu-hingga-rekor-donasi-kripto-untuk-trump/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/sejumlah-agenda-krusial-bakal-gerakkan-pasar-kripto-dalam-minggu-ini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/chatgpt-4-o-beberkan-alasan-kenapa-blackrock-hanya-fokus-pada-bitcoin-dan-ethereum/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rco-finance-rcof-dan-shiba-inu-shib-bersaing-dalam-peningkatan-laju-pembakaran-akselerasi-harga-segera-terjadi/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-breakout-ini-tanda-tanda-altseason-yang-harus-diperhatikan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-bergerak-naik-di-tengah-sentimen-positif-apakah-69000-akan-kuat/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-alami-penurunan-tajam-saat-upaya-tembus-us-70-000-gagal/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-bakal-raih-lagi-ath-ini-tips-trading-ideal-menurut-analis/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/whale-cardano-dan-bonk-memperhatikan-meme-coin-ai-raboo/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/apa-saja-yang-anda-lewatkan-saat-absen-24-jam-di-pasar-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-token-unlock-yang-harus-diwaspadai-prime-op-yfi-pendle-1inch/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ceo-blockdag-antony-turner-mantan-co-founder-swissone-capital-menargetkan-us-600-juta-mengarahkan-investor-menjauh-dari-tren-sol-dan-dot/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/payout-mt-gox-sudah-berada-di-tangan-para-kreditur-pasar-kripto-siap-berguncang/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ethereum-terperosok-pasca-alami-kenaikan-signifikan-ada-apa/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pemerintahan-rusia-kemungkinan-akan-masuk-ke-ranah-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-kripto-potensial-dalam-kategori-defi-jto-aave-jup-dodo-uma/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/wawancara-ceo-blockdag-dan-tim-ama-tingkatkan-kepercayaan-membuat-roi-kelas-terkemuka-menjadi-mungkin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/jangan-ketinggalan-sejumlah-hal-besar-terjadi-pada-pasar-kripto-12-jam-lalu/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-bull-run-di-depan-mata-analis-ini-urai-kaitannya-dengan-aset-emas/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/koin-meme-iq-capai-kapitalisasi-pasar-us-5-juta-dalam-24-jam-kok-bisa/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-airdrop-potensial-dalam-waktu-dekat-untuk-para-pejuang-airdrop-2/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/wawancara-ceo-ungkap-informasi-utama-lonjakan-presale-us-64-juta-blockdag-wawasan-icp-berita-ton/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/microstrategy-beli-bitcoin-lagi-ini-jumlah-total-kepemilikannya-sekarang/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/apakah-bitcoin-akan-semakin-turun-ini-level-yang-perlu-diwaspadai-investor/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/analisa-shiba-inu-aptos-dan-raboo-coin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ini-sejumlah-analisa-terkait-puncak-harga-bitcoin-selanjutnya-dari-trader-hingga-peneliti/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/apakah-optimism-op-masih-layak-dibeli-melihat-analisis-data-onchain/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/harga-bitcoin-terjun-bebas-analis-peringatkan-potensi-penurunan-lebih-lanjut/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/empat-pengusaha-kripto-terkaya-dan-kerajaan-nya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ekonomi-as-di-ujung-tanduk-ekonom-ini-prediksi-resesi-dan-inflasi-tinggi/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-jatuh-hingga-kisaran-us-60-000-ini-biang-keladinya-menurut-trader-papan-atas/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/token-bertema-anjing-tumpang-tindih-dengan-tren-meme-ai-kelinci-bernama-raboo-sudahkah-anda-ikut/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/binance-sambut-hamster-kombat-antusiasme-meledak-di-pasar-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ai-ungkap-alasan-solana-lebih-bagus-daripada-cardano/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/apakah-ada-kesempatan-pulih-bagi-bitcoin-setelah-anjlok-di-bawah-us-60-000/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/kenapa-harga-bitcoin-turun-ke-53-000-hari-ini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/penambang-bitcoin-kini-dihadapkan-pada-sejumlah-tantangan-baru/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ripple-bersiap-jual-200-juta-xrp-ada-apa/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/3-faktor-penyebab-koreksi-bitcoin-ke-daerah-50-000-dalam-3-hari-berturut-turut/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/seluruh-aset-kripto-ambruk-dalam-hitungan-jam-simak-kilasan-info-nya-di-sini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/harga-bitcoin-btc-turun-di-bawah-us-60-000-diikuti-ripple-xrp-saat-tekanan-bearish-meningkat-pada-token-perusahaan-pembayaran/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bernstein-pasar-kripto-tertekan-tapi-pelakunya-bukan-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/akankah-level-di-bawah-us-50-000-jadi-petaka-untuk-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/apakah-dot-masih-layak-dibeli-melihat-analisis-data-transaksi-polkadot/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/analis-sebut-bitcoin-membentuk-pola-rising-wedge-apa-artinya-bagi-investor/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/whale-kripto-tak-terpengaruh-oleh-penurunan-aliran-keluar-spot-ethereum-pembelian-besar-eth-terus-berlanjut/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/qcp-capital-vs-crypnuevo-dalam-pandangan-terkait-potensi-pergerakan-harga-bitcoin-selanjutnya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/investor-ini-tuding-jepang-dan-amerika-jadi-penyebab-utama-guncangan-hebat-di-pasar-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/apakah-link-masih-layak-dibeli-meneliti-data-transaksi-chainlink/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-turun-di-bawah-us-50-000-simak-pendapat-para-ahli-berikut/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/apa-itu-solayer-proyek-restaking-terbaru-yang-didukung-binance-labs/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/koin-meme-baru-neiro-siap-hebohkan-pasar-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-token-unlock-yang-harus-diwaspadai-gmx-imx-xai-fort-glmr/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rebound-besar-analis-ini-perkirakan-harga-bitcoin-masih-bisa-naik-ke-us-90-000/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/trader-dogecoin-doge-legendaris-mengatakan-token-ethereum-seharga-us-00344-ini-akan-naik/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/arthur-hayes-klaim-target-harga-bitcoin-dan-ethereum-saat-carut-marut-ekonomi-amerika/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/airdrop-hamster-kombat-diisukan-jadi-yang-terbesar-sepanjang-sejarah-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/harga-bitcoin-bergerak-naik-setelah-turun-drastis-semua-volume-beli-berasal-dari-whale-institusional/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/mulai-dari-legalisasi-kripto-di-rusia-hingga-pertemuan-bersama-gedung-putih-ini-deretan-info-kripto-dalam-24-jam/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/kenapa-neiro-mona-dan-shibtc-jadi-primadona-di-kalangan-koin-meme/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pasar-altcoin-di-ambang-pemulihan-menurut-analis-benjamin-cowen/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/saat-pasar-kripto-anjlok-trader-ini-empat-kripto-untuk-tetap-dulang-cuan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rangkuman-berita-cryptoharian-binance-sambut-hamster-kombat-hingga-pasar-altcoin-di-ambang-pemulihan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ekonomi-dunia-diperkirakan-hancur-di-tahun-2026-ini-deretan-penyebabnya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/6-koin-yang-memiliki-agenda-minggu-ini-bitcoin-hingga-sandbox/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bagaimana-harga-bitcoin-ditentukan-ini-jawaban-lengkapnya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/apakah-shiba-inu-bisa-bangkit-kembali-setelah-kejatuhan-besarnya-baru-baru-ini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/youssef-khaoulaj-ahli-metaverse-bersertifikat-memimpin-blockdag-lebih-banyak-tentang-perkiraan-harga-cardano-ada-uniswap-uni/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/volatilitas-berlanjut-menjelang-publikasi-data-inflasi-amerika-bitcoin-turun-ke-bawah-60-000/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/sec-novatech-lakukan-penipuan-kripto-us-650-juta-jerat-ribuan-investor/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/solana-sol-cardano-ada-dan-token-defi-ethereum-eth-ini-tampaknya-akan-bangkit-saat-bear-trap-bitcoin-selesai/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/hamster-kombat-berani-tolak-pemodal-raksasa-ini-alasannya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pekan-ini-diprediksi-jadi-momen-krusial-bagi-bitcoin-saat-harga-sentuh-kisaran-us-57-000/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-token-unlock-yang-harus-diwaspadai-crv-pendle-sand-cyber-strk/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/cpi-amerika-jadi-penentu-arah-harga-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/sejumlah-analis-perkirakan-harga-bitcoin-bakal-capai-nilai-enam-digit/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/banyak-prediksi-meleset-kini-giliran-arthur-hayes-ungkap-kapan-altcoin-akan-berjaya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/indonesia-siap-memimpin-kedaulatan-digital-melalui-bali-blockchain-summit-2024-dan-web3-hackathon/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/harga-bitcoin-kembali-naik-ke-61-000-didukung-oleh-aksi-beli-whale-dan-data-inflasi-yang-lebih-rendah/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-koreksi-5-di-tengah-publikasi-data-ekonomi-as-kesempatan-untuk-untung/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/mengapa-bank-besar-seperti-goldman-sachs-mulai-melirik-bitcoin/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/altcoin-sherpa-bitcoin-berada-di-persimpangan-siap-siap-untuk-pergerakan-besar/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bagaimana-dengan-blockdag-setelah-harga-binance-coin-dan-protokol-near-jatuh/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/smart-money-mengumpulkan-jutaan-dalam-cardano-ada-pepe-dan-shiba-inu-killer-senilai-00344-ini-untuk-keuntungan-4600/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/5-airdrop-potensial-dalam-untuk-para-pencari-cuan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/kilas-balik-24-jam-terakhir-dalam-perkembangan-terbaru-di-dunia-kripto/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-naik-dari-harga-us-58-000-sinyal-bahaya-atau-peluang/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/trader-veteran-yang-membeli-dogecoin-di-awal-mengatakan-token-ethereum-ini-akan-mengalami-kenaikan/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/ai-ini-prediksi-harga-ethereum-di-penghujung-bulan-berikut-targetnya/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bangun-kekayaan-real-estat-anda-dengan-game-clickcity-senilai-us-1-juta-dari-lufina/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/7-alasan-kenapa-bitcoin-anjlok-sebentar-lagi-menurut-analis-berikut/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/pasar-kripto-semakin-kemari-semakin-kehilangan-marwah/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/rangkuman-berita-cryptoharian-perkiraan-ekonomi-hancur-hingga-prediksi-ethereum-akhir-agustus/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/simak-apa-saja-yang-akan-gerakkan-pasar-kripto-dalam-pekan-ini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/analis-kripto-benjamin-cowen-yakin-dominasi-bitcoin-akan-capai-60-persen/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/whale-bergerak-litecoin-dan-dogecoin-jadi-fokus-utama/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/hati-hati-50-persen-koin-meme-mengalami-kerugian/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
    {
        'url': 'https://cryptoharian.com/bitcoin-jatuh-di-bawah-60-000-apa-arti-dari-koreksi-ini/',
        'title_tag': 'h1',
        'title_class': 'entry-title',
        'date_tag': 'div',
        'date_class': 'meta-item meta-date',
        'content_tag': 'div',
        'content_class': 'entry-content'
    },
]

all_articles = []

# Collect and summarize data from each website
for website in websites:
    article = scrape_website(
        website['url'], 
        website['title_tag'], 
        website['title_class'], 
        website['date_tag'], 
        website['date_class'], 
        website['content_tag'], 
        website['content_class']
    )
    all_articles.append(article)

# Save summarized data to CSV file
csv_file = './preprocessed/cryptoharian.csv'
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['Title', 'Date', 'Summary'])
    writer.writeheader()
    for article in all_articles:
        writer.writerow(article)

# Print the contents of the CSV file
print(f"Data has been saved to {csv_file}")

df = pd.read_csv(csv_file)
print(df)

c:\Users\Reyhan\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\Reyhan\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


Data has been saved to ./preprocessed/cryptoharian.csv
                                                 Title              Date  \
0    Top 3 Marketplace Bitcoin NFT yang Harus Diket...     Juni 26, 2024   
1    SEC Diperkirakan Bakal Setujui ETF Ethereum Sp...     Juni 27, 2024   
2    3 Kripto Yang Harus Dijual Karena Ada Berita T...     Juni 27, 2024   
3    Pendiri DiamondSwap: Saya Masih Bullish Terhad...     Juni 27, 2024   
4    Ingin Mulai Perjalanan Investasi Kripto? Simak...     Juni 27, 2024   
..                                                 ...               ...   
253  Simak Apa Saja yang Akan Gerakkan Pasar Kripto...  Agustus 19, 2024   
254  Analis Kripto Benjamin Cowen Yakin Dominasi Bi...  Agustus 19, 2024   
255  Whale Bergerak, Litecoin dan Dogecoin Jadi Fok...  Agustus 19, 2024   
256  Hati-Hati! 50 Persen Koin Meme Mengalami Kerugian  Agustus 19, 2024   
257  Bitcoin Jatuh di Bawah $60.000: Apa Arti dari ...  Agustus 19, 2024   

                                